In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import utils
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
data_dir = '../input'
!ls ../input

sample_submission_v2.csv  test-tif-v2		    train-jpg	  train_v2.csv
test-jpg		  test_v2_file_mapping.csv  train-tif-v2


In [3]:
train_label = pd.read_csv(os.path.join(data_dir, 'train_v2.csv'))

In [4]:
labels_str = 'agriculture, artisinal_mine, bare_ground, blooming, blow_down, clear, cloudy, conventional_mine, cultivation, habitation, haze, partly_cloudy, primary, road, selective_logging, slash_burn, water'
labels = labels_str.split(', ')

In [5]:
label_map = {x: labels.index(x) for x in labels}
label_map

{'agriculture': 0,
 'artisinal_mine': 1,
 'bare_ground': 2,
 'blooming': 3,
 'blow_down': 4,
 'clear': 5,
 'cloudy': 6,
 'conventional_mine': 7,
 'cultivation': 8,
 'habitation': 9,
 'haze': 10,
 'partly_cloudy': 11,
 'primary': 12,
 'road': 13,
 'selective_logging': 14,
 'slash_burn': 15,
 'water': 16}

In [6]:
def tags_to_vec(tags):
    tags_list = tags.split(' ')
    vec = np.zeros(17)
    for t in tags_list:
        vec[label_map[t]] = 1
    return vec

In [7]:
train_label['y'] = train_label.tags.apply(tags_to_vec)

In [8]:
pd.set_option('max_colwidth',-1)
train_label

,image_name,tags,y
0,train_0,haze primary,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]"
1,train_1,agriculture clear primary water,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]"
2,train_2,clear primary,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]"
3,train_3,clear primary,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]"
4,train_4,agriculture clear habitation primary road,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0]"
5,train_5,haze primary water,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]"
6,train_6,agriculture clear cultivation primary water,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]"
7,train_7,haze primary,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]"
8,train_8,agriculture clear cultivation primary,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]"
9,train_9,agriculture clear cultivation primary road,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0]"


In [9]:
X_train = []
y_train = []
N_train_limit = 2000e9
i=0
for idx, row in tqdm(train_label.iterrows(), total=min(N_train_limit, train_label.shape[0])):
    image = cv2.imread(os.path.join('../input', 'train-jpg', '{}.jpg'.format(row['image_name'])))
    #image = image[:,:,::-1]
    #plt.imshow(image)
    X_train.append(image)
    y_train.append(row['y'])
    i+=1
    if i==N_train_limit:
        break

100%|██████████| 40479/40479 [01:05<00:00, 614.05it/s]


In [10]:
X_train = np.stack(X_train, axis=0)
X_train.shape

(40479, 256, 256, 3)

In [11]:
y_train = np.stack(y_train, axis=0)
y_train.shape

(40479, 17)

In [12]:
X_train = utils.preprocess_data(X_train, use_grayscale=True, keep_original=False, normed=True)

  0%|          | 131/40479 [00:00<00:30, 1301.58it/s]

Input shapes:  (40479, 256, 256, 3)
Using gray


100%|██████████| 40479/40479 [00:35<00:00, 1142.78it/s]


Output shapes:  (40479, 256, 256, 1)


In [13]:
xtrain, xvalid, ytrain, yvalid = train_test_split(X_train, y_train, test_size = 0.2)
print(xtrain.shape, xvalid.shape, ytrain.shape, yvalid.shape)

(32383, 256, 256, 1) (8096, 256, 256, 1) (32383, 17) (8096, 17)


In [14]:
import gc
del X_train, y_train
gc.collect()

226

In [15]:
gc.collect()
results = utils.train_model(xtrain, ytrain, xvalid, yvalid, xvalid, yvalid, resuming=True, batch_size=64)

========= train_model() arguments: ==========
resuming = True
model = <function lenet at 0x7f8c18803158>
model_params = {'num_classes': 17, 'name': 'orig_lenet', 'fc3_size': 120, 'conv2_p': 0.95, 'conv1_k': 5, 'conv2_d': 16, 'fc4_p': 0.5, 'fc3_p': 0.5, 'conv1_p': 0.95, 'conv2_k': 5, 'model_name': 'lenet', 'fc4_size': 84, 'conv1_d': 6}
learning_rate = 0.001
max_epochs = 1001
batch_size = 64
early_stopping_enabled = True
early_stopping_patience = 10
log_epoch = 1
print_epoch = 1
orig_lenet
lenet__conv1_d_6_conv1_k_5_conv1_p_0.95_conv2_d_16_conv2_k_5_conv2_p_0.95_fc3_p_0.5_fc3_size_120_fc4_p_0.5_fc4_size_84
6f1fbf83feb9ed73
model dir: /home/dingran/github/kaggle-amazon/code/models/6f1fbf83feb9ed73
{'num_classes': 17, 'fc3_size': 120, 'conv2_p': 0.95, 'conv1_k': 5, 'conv2_d': 16, 'fc4_p': 0.5, 'fc3_p': 0.5, 'conv1_p': 0.95, 'conv2_k': 5, 'model_name': 'lenet', 'fc4_size': 84, 'conv1_d': 6}
lenet pool2 reshaped size:  [None, 65536]
<tf.Variable 'conv1/weights:0' shape=(5, 5, 1, 6) dtype=flo

100%|██████████| 506/506 [01:07<00:00,  4.09it/s]


-------------- EPOCH    0/1001 --------------
     Train loss: 0.20339487, f2beta: 78.00%
Validation loss: 0.20402244, f2beta: 78.24%
      Best loss: inf at epoch 0
   Elapsed time: 00:01:52
      Timestamp: 2017/07/11 22:12:38


MemoryError: 

In [ ]:
import glob
X_test = []
test_file_list = glob.glob('../input/test-jpg/*.jpg')
test_filenames = []
for t in tqdm(test_file_list):
    filename = os.path.basename(t).replace('.jpg', '')
    test_filenames.append(filename)
    image = cv2.imread(t)
    image = utils.preprocess_single_image(image, use_grayscale=True, keep_original=False, normed=True)
    X_test.append(image)

In [ ]:
del X_train, y_train
import gc
gc.collect()

In [ ]:
X_test = np.stack(X_test, axis=0)
X_test.shape

In [ ]:
#y_pred, sigmoid_pred = utils.train_model(xtrain, ytrain, xvalid, yvalid, xvalid, yvalid, max_epochs=-2, resuming=True)
y_pred, sigmoid_pred = utils.train_model(None, None, None, None, X_test, None, max_epochs=-2, resuming=True)

In [ ]:
def map_predictions(predictions, labels_map, thresholds=np.ones(17)*0.2):
    predictions_labels = []
    for prediction in predictions:
        labels = [labels_map[i] for i, value in enumerate(prediction) if value > thresholds[i]]
        predictions_labels.append(labels)

    return predictions_labels

In [ ]:
predicted_labels = map_predictions(sigmoid_pred, labels)

In [ ]:
predicted_labels_str = [' '.join(x) for x in predicted_labels]

In [ ]:
df = pd.DataFrame({'image_name': test_filenames, 'tags': predicted_labels_str})
df.to_csv('../output/2nd_try.csv', index=False)